# Lucid Feature Visualizations

## Install and Import

### Install Lucid

In [1]:
!pip install --quiet lucid
!pip freeze | grep "lucid"

lucid==0.3.8


### Import Libraries

In [0]:
from progressbar import ProgressBar, Bar, Percentage
from imageio import imwrite

import lucid.modelzoo.vision_models as models
import lucid.optvis.objectives as objectives
import lucid.optvis.transform as transform
import lucid.optvis.render as render
import lucid.optvis.param as param
import numpy as np
import sys
import os
import gc

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf

## Init

In [0]:
def make_directories():
    if not os.path.exists('bin'):
        os.mkdir('bin')

    if not os.path.exists(os.path.join('bin', 'shapes')):
        os.mkdir(os.path.join('bin', 'shapes'))

In [0]:
make_directories()

## Model Selector

In [0]:
def get_model(model_name):
    model = None
    if model_name == 'mobile-net-v1':
        model = models.MobilenetV1_slim()
    elif model_name == 'mobile-net-v2':
        model = models.MobilenetV2_14_slim()
    elif model_name == 'vgg-16':
        model = models.VGG16_caffe()
    elif model_name == 'vgg-19':
        model = models.VGG19_caffe()
    elif model_name == 'alexnet':
        model = models.AlexNet()
    elif model_name == 'resnet-v1-50':
        model = models.ResnetV1_50_slim()
    elif model_name == 'resnet-v1-101':
        model = models.ResnetV1_101_slim()
    elif model_name == 'resnet-v1-152':
        model = models.ResnetV1_152_slim()
    elif model_name == 'resnet-v2-50':
        model = models.ResnetV2_50_slim()
    elif model_name == 'resnet-v2-101':
        model = models.ResnetV2_101_slim()
    elif model_name == 'resnet-v2-152':
        model = models.ResnetV2_152_slim()
    elif model_name == 'inception-v1':
        model = models.InceptionV1_slim()
    elif model_name == 'inception-v2':
        model = models.InceptionV2_slim()
    elif model_name == 'inception-v3':
        model = models.InceptionV3_slim()
    elif model_name == 'inception-v4':
        model = models.InceptionV4_slim()
    elif model_name == 'inception-resnet-v2':
        model = models.InceptionResnetV2_slim()
    model.load_graphdef()
    return model

## Sample Generator

### Configurations

In [0]:
MODEL_NAME = 'vgg-16'
LEARNING_RATE = 0.05
ROBUSTNESS  = True
DECORRELATE = True
L1_LAMBDA = 0.001
TV_LAMBDA = 0.001
BLUR_LAMBDA = 0.125
THRESHOLD = (512,)

### Select Model

In [0]:
model = get_model(MODEL_NAME)

### Model Layers

In [0]:
layers = list(map(lambda x: (x.name, x.depth), filter(lambda x: x.tags=={'conv'}, model.layers)))

In [9]:
print('{:70} {}'.format('Layer Name', 'Layer Depth'))
for lname, depth in layers:
    print('{:70} {}'.format(lname, depth))

Layer Name                                                             Layer Depth
conv1_1/conv1_1                                                        64
conv1_2/conv1_2                                                        64
conv2_1/conv2_1                                                        128
conv2_2/conv2_2                                                        128
conv3_1/conv3_1                                                        256
conv3_2/conv3_2                                                        256
conv3_3/conv3_3                                                        256
conv4_1/conv4_1                                                        512
conv4_2/conv4_2                                                        512
conv4_3/conv4_3                                                        512
conv5_1/conv5_1                                                        512
conv5_2/conv5_2                                                        512
conv5_3/conv5_3    

In [0]:
layers = dict(layers)

### Common Configurations

In [0]:
optimizer = tf.train.AdamOptimizer(LEARNING_RATE)
if ROBUSTNESS:
    transforms = transform.standard_transforms
else:
    transforms = []

### Image Processing Helper Function

In [0]:
def deprocess_image(x):
    return np.clip(((x - np.min(x)) / (np.max(x) - np.min(x))) * 255.0, 0, 255).astype('uint8')

### Main Function

In [0]:
def main(layer_id, layer_name, level):
    np.random.seed(42)
    
    path = os.path.join('bin', MODEL_NAME)
    if not os.path.exists(path):
        os.mkdir(path)

    path = os.path.join(path, layer_id)
    if not os.path.exists(path):
        os.mkdir(path)
        os.mkdir(os.path.join(path, 'neuron'))
        os.mkdir(os.path.join(path, 'positive'))
        os.mkdir(os.path.join(path, 'negative'))
        os.mkdir(os.path.join(path, 'diverse'))
    
    path_neuron = os.path.join(path, 'neuron')
    path_positive = os.path.join(path, 'positive')
    path_negative = os.path.join(path, 'negative')
    path_diverse = os.path.join(path, 'diverse')
    
    bar = ProgressBar(maxval=layers[layer_name],
                      widgets=[Bar('=', '[', ']'), ' ', Percentage()]).start()
    
    for i in range(52, 64): # layers[layer_name]):
        imgs = None
        if level==1:
            param_neuron = lambda: param.image(128, fft=DECORRELATE, decorrelate=DECORRELATE)
            obj_neuron = objectives.neuron(layer_name=layer_name, channel_n=i)
            imgs = render.render_vis(model=model, objective_f=obj_neuron,
                                 param_f=param_neuron, optimizer=optimizer,
                                 transforms=transforms, thresholds=THRESHOLD,
                                 verbose=False, use_fixed_seed=True)[0]
            imwrite(os.path.join(path_neuron, 'nrn-{:04d}.png'.format(i)), deprocess_image(imgs[0]))
            del param_neuron
        elif level==2:
            param_pos_neg = lambda: param.image(128, fft=DECORRELATE, decorrelate=DECORRELATE, batch=2)
            obj_pos_neg = objectives.channel(layer=layer_name, n_channel=i, batch=1)
            obj_pos_neg -= objectives.channel(layer=layer_name, n_channel=i, batch=0)
            imgs = render.render_vis(model=model, objective_f=obj_pos_neg,
                                 param_f=param_pos_neg, optimizer=optimizer,
                                 transforms=transforms, thresholds=THRESHOLD,
                                 verbose=False, use_fixed_seed=True)[0]
            imwrite(os.path.join(path_negative, 'neg-{:04d}.png'.format(i)), deprocess_image(imgs[0]))
            imwrite(os.path.join(path_positive, 'pos-{:04d}.png'.format(i)), deprocess_image(imgs[1]))
            del param_pos_neg
        elif level==3:
            param_diverse = lambda: param.image(64, fft=DECORRELATE, decorrelate=DECORRELATE, batch=4)
            obj_diverse = objectives.channel(layer=layer_name, n_channel=i)
            obj_diverse -= objectives.diversity(layer_name)
            imgs = render.render_vis(model=model, objective_f=obj_diverse,
                                 param_f=param_diverse, optimizer=optimizer,
                                 transforms=transforms, thresholds=THRESHOLD,
                                 verbose=False, use_fixed_seed=True)[0]
            img = np.zeros((128, 128, 3))
            img[:64, :64] = imgs[0]
            img[:64, 64:] = imgs[1]
            img[64:, :64] = imgs[2]
            img[64:, 64:] = imgs[3]
            imwrite(os.path.join(path_diverse, 'div-{:04d}.png'.format(i)), deprocess_image(img))
            del param_diverse
            del img
        bar.update(i + 1)
        del imgs

    bar.finish()

In [14]:
# main('conv1_1', 'conv1_1/conv1_1', 1)
# main('conv1_1', 'conv1_1/conv1_1', 2)
main('conv1_1', 'conv1_1/conv1_1', 3)

[                                                                        ] N/A%WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
[========================================================================] 100%


In [0]:
_ = gc.collect()

In [0]:
# !rm -rf bin/mobile-net-v1

In [18]:
!cd bin/vgg-16; zip -r vgg-16@conv1_1 conv1_1; cd ../..

  adding: conv1_1/ (stored 0%)
  adding: conv1_1/diverse/ (stored 0%)
  adding: conv1_1/diverse/div-0014.png (stored 0%)
  adding: conv1_1/diverse/div-0050.png (deflated 0%)
  adding: conv1_1/diverse/div-0061.png (stored 0%)
  adding: conv1_1/diverse/div-0039.png (stored 0%)
  adding: conv1_1/diverse/div-0024.png (stored 0%)
  adding: conv1_1/diverse/div-0042.png (stored 0%)
  adding: conv1_1/diverse/div-0019.png (stored 0%)
  adding: conv1_1/diverse/div-0035.png (stored 0%)
  adding: conv1_1/diverse/div-0027.png (stored 0%)
  adding: conv1_1/diverse/div-0056.png (stored 0%)
  adding: conv1_1/diverse/div-0033.png (stored 0%)
  adding: conv1_1/diverse/div-0049.png (stored 0%)
  adding: conv1_1/diverse/div-0034.png (stored 0%)
  adding: conv1_1/diverse/div-0017.png (stored 0%)
  adding: conv1_1/diverse/div-0012.png (stored 0%)
  adding: conv1_1/diverse/div-0031.png (stored 0%)
  adding: conv1_1/diverse/div-0057.png (stored 0%)
  adding: conv1_1/diverse/div-0047.png (stored 0%)
  adding: 